In [1]:
import pandas as pd
import os

In [2]:
# 데이터 로드
search_path = os.getcwd()
file_tr = "Q2_train.csv"
file_te = "Q2_test.csv"
file_samp = "Q2_label_sample.csv"

for root, dirs, files in os.walk(search_path):
    if file_tr in files:
        file_path = os.path.join(root, file_tr)
        train_df = pd.read_csv(file_path)
    if file_te in files:
        file_path = os.path.join(root, file_te)
        test_df = pd.read_csv(file_path)
    if file_samp in files:
        file_path = os.path.join(root, file_samp)
        label = pd.read_csv(file_path)

In [13]:
alarmmsg_train = pd.DataFrame({'alarmmsg_original': train_df.alarmmsg_original.unique()})
alarmmsg_test = pd.DataFrame({'alarmmsg_original': test_df.alarmmsg_original.unique()})
alarmmsg = pd.concat([alarmmsg_train, alarmmsg_test])
alarmmsg.sort_values('alarmmsg_original', inplace=True)

In [15]:
alarmmsg.head()

,alarmmsg_original
24,48V-FAIL
69,48V-FAIL
46,AUTONEGO_MISMATCH
8,AUTONEGO_MISMATCH
66,Abnormal Condition: Laser Forced


In [16]:
# 메세지 전처리 함수
def ppr_msg(s):
    s = s.lower()
    s = s.replace('(', '').replace(')', '') # 괄호 제거
    s = s.replace(':', '').replace('+', '') # ':', '+' 제거
    s = s.replace('-', ' ').replace('_', ' ') # '-', '_' 구분자 공백으로 바꾸기

    # 의미 없는 말 제거, ABC 기지국 보편적인 약어 풀이를 사용하여
    # 의미에 따라 경보 메세지 전처리
    s = s.replace('remote defect indication', 'rdi') # ex. remote rdi로 바뀔 것, defect와 indication은 의미 없는 말
    s = s.replace('remote fault indication', 'remote rfi')
    s = s.replace('fan 48v fail', 'fan fail')
    s = s.replace('48v fail', 'input pwr conn mis')
    s = s.replace('abnormal condition laser forced', 'laser force')
    s = s.replace('battery', 'batt')
    s = s.replace('dc input a fail', 'dc input fail')
    s = s.replace('dcc', 'dcc communicate channel')
    s = s.replace('dcn', 'dcc comunication network') #communicate라고 해도 결과 안달라지는지 확인
    s = s.replace('down los', 'down loss signal')
    s = s.replace('ether', 'eth')
    s = s.replace('llcf', 'loss carry forward')
    s = s.replace(' fault', '')
    s = s.replace('failure', 'fail')
    s = s.replace(' alarm', '')
    s = s.replace('improper removal', 'remove')
    s = s.replace(' defect', '')
    s = s.replace('power', 'pwr')
    s = s.replace('links', 'link')
    s = s.replace(' with', '')
    s = s.replace('loss of connectivity', 'loss connectivity')
    s = s.replace('loss of signal', 'loss signal')
    s = s.replace('mep lsp loc', 'mep lsp loss connectivity')
    s = s.replace('lsp loc', 'lsp oam loss connectivity')
    s = s.replace('manual management removal', 'mng remove')
    s = s.replace('lsp loc', 'lsp loss connectivity')
    s = s.replace('rdi', 'remote rdi')
    s = s.replace('csf', 'client signal fail')
    s = s.replace('pwe ', 'pw eth ')
    s = s.replace('pw eth loc', 'pw eth loss connectivity')
    s = s.replace(' sfp', '')
    s = s.replace(' via', '')
    s = s.replace(' primary', '')
    s = s.replace('reachable', 'reach')
    s = s.replace(' of', '')
    s = s.replace('oamloss', 'oam loss')
    s = s.replace('opt los', 'opt loss signal')
    s = s.replace('opt mis', 'opt mismatch')
    s = s.replace('trk conn mis', 'trk conn mismatch')
    s = s.replace('portshutdown', 'port down')
    s = s.replace('protectionswitched', 'protect switch')
    s = s.replace('pw loc', 'pw loss continuity')
    s = s.replace('restarted', 'restart')
    s = s.replace('stm1 los', 'sync transport module loss signal')
    s = s.replace('lof', 'loss frame')
    s = s.replace('rmt', 'remote')
    s = s.replace('unexpected', 'unexpect')
    s = s.replace(' by', '')
    s = s.replace('ipc fail', 'ipc communicate fail')
    s = s.replace('unit rem', 'unit remove')
    s = s.replace('raise', '')
    s = s.replace('synchronization', 'sync')
    s = s.replace(' signals', '')

    return s

In [17]:
alarmmsg['alarmmsg_edited'] = alarmmsg['alarmmsg_original'].apply(ppr_msg)

In [18]:
alarmmsg

,alarmmsg_original,alarmmsg_edited
24,48V-FAIL,input pwr conn mis
69,48V-FAIL,input pwr conn mis
46,AUTONEGO_MISMATCH,autonego mismatch
8,AUTONEGO_MISMATCH,autonego mismatch
66,Abnormal Condition: Laser Forced,laser force
...,...,...
59,UNIT_INIT (BY INSLOT),unit init inslot
60,UNIT_INIT (BY RESET),unit init reset
61,UNIT_OUT,unit out
47,Underlying Resource Unavailable,underlying resource unavailable


In [20]:
alarmmsg.to_excel("alarmmsg_edited.xlsx", index=False)